In [20]:
import json
from typing import List, Set, Tuple
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from urllib.request import urlopen
from urllib.parse import urlencode
import xml.etree.ElementTree as ET
import time
import re
import os
import sys
from abstract_retriever import AbstractRetriever
sys.path.append(".")
from apis.gpt_api import get_gpt_response

In [21]:
SIMILARITY_THRESHOLD = 5

db_file = "pubmed_index/pubmed_data.db"
h5_file = "pubmed_index/pubmed_embeddings.h5"

os.environ['CUDA_VISIBLE_DEVICES'] = '6'
retriever = AbstractRetriever(h5_file, db_file, chunk_size=250000, use_cuda=True)

<All keys matched successfully>


In [3]:
def filter_similar_sets(concept_sets: List[Set[str]], similarity_threshold: int = 5) -> List[Set[str]]:
    """Filter out highly similar concept sets based on the concept multihot vector."""
    vectorizer = CountVectorizer(vocabulary=list(set(concept for concept_set in concept_sets for concept in concept_set)))
    concept_vectors = vectorizer.transform([' '.join(concept_set) for concept_set in concept_sets])
    filtered_sets = []
    processed_indices = set()

    for i, concept_set in enumerate(concept_sets):
        if i not in processed_indices and len(concept_set) > 10:
            similar_indices = [j for j, other_set in enumerate(concept_sets) if i != j and len(other_set) > 10 and
                               len(concept_set.symmetric_difference(other_set)) < similarity_threshold]
            filtered_sets.append(concept_set)
            processed_indices.update(similar_indices)
        elif i not in processed_indices:
            filtered_sets.append(concept_set)

    return filtered_sets

In [10]:
def extract_relationships(text: str, concepts: List[str]) -> str:
    """Extract relationships from text using LLM."""
    example = """Example:
    Text:
    Asthma is a chronic respiratory condition characterized by inflammation and narrowing of the airways, leading to breathing difficulties. Common symptoms include wheezing, coughing, shortness of breath, and chest tightness. Triggers can vary but often include allergens, air pollution, exercise, and respiratory infections. Management typically involves a combination of long-term control medications, such as inhaled corticosteroids, and quick-relief medications like short-acting beta-agonists. Recent research has focused on personalized treatment approaches, including biologics for severe asthma and the role of the microbiome in asthma development and progression. Proper inhaler technique and adherence to medication regimens are crucial for effective management. Asthma action plans, developed in partnership with healthcare providers, help patients manage symptoms and exacerbations.

    Concepts: [asthma, inflammation, airways, wheezing, coughing, inhaled corticosteroids, short-acting beta-agonists, allergens, respiratory infections]

    Extracted triples:
    [[asthma, is a, chronic respiratory condition], [asthma, characterized by, inflammation of airways], [inflammation, causes, narrowing of airways], [narrowing of airways, leads to, breathing difficulties], [wheezing, is a symptom of, asthma], [coughing, is a symptom of, asthma], [allergens, can trigger, asthma], [respiratory infections, can trigger, asthma], [inhaled corticosteroids, used for, long-term control of asthma], [short-acting beta-agonists, provide, quick relief in asthma]]
    """

    prompt = f"""Given a medical text and a list of important concepts, extract relevant relationships between the concepts from the text (if present). For each triple, if an entity matches one of the given concepts, replace the entity with the exact concept term.

    Focus on generating high-quality triples closely related to the provided concepts. Aim to extract at most 10 triples for each text. Each triple should follow this format: [ENTITY1, RELATIONSHIP, ENTITY2]. Ensure the triples are informative and logically sound.

    {example}

    Text:
    {text}

    Concepts: {concepts}

    Extracted triples:
    """
    # print(f'{prompt=}')
    # print(f'{concepts=}')
    # print(f'{text=}')
    # print(f'{concepts=}')

    # print('DONE')
    response = get_gpt_response(prompt=prompt)
    # print('\n\n---------RESPONSE:-------',response)
    return response

In [11]:
def parse_triples(response: str, concept_set: Set[str]) -> List[Tuple[str, str, str]]:
    """Parse triples from LLM response."""
    pattern = r"\[(.*?)\]"
    matches = re.findall(pattern, response)
    
    triples = []
    for match in matches:
        triple = match.split(', ')
        if len(triple) == 3:
            entity1, relation, entity2 = triple
            entity1 = entity1.replace("[", "").replace("]", "")
            entity2 = entity2.replace("[", "").replace("]", "")
            triples.append((entity1, relation, entity2))
    print(triples)
    return triples

In [23]:
# Load the all_visit_concepts from the JSON file
with open("/kg_construct/all_visit_concepts_DN.json", "r") as f:
    all_visit_concepts = [set(visit) for visit in json.load(f)]

if os.path.exists(f"/project/biocomplexity/hht9zt/KARE/kg_construct/filtered_concept_sets_{SIMILARITY_THRESHOLD}_DN.json"):
    print("Loading filtered concept sets from file...")
    with open(f"/project/biocomplexity/hht9zt/KARE/kg_construct/filtered_concept_sets_{SIMILARITY_THRESHOLD}_DN.json", "r") as f:
        filtered_concept_sets = json.load(f)
else:
    print("Filtering similar concept sets...")
    filtered_concept_sets = filter_similar_sets(all_visit_concepts)

with open(f"/project/biocomplexity/hht9zt/KARE/kg_construct/filtered_concept_sets_{SIMILARITY_THRESHOLD}_DN.json", "w") as f:
    json.dump([list(s) for s in filtered_concept_sets], f)

# Filter out highly similar concept sets
filtered_concept_sets = filter_similar_sets(all_visit_concepts)

In [ ]:
# def build_kg_for_concepts(concept_sets: List[Set[str]]) -> List[Tuple[str, str, str]]:
"""Build knowledge graph for medical concepts using PubMed abstracts and LLM."""
    
concept_sets = filtered_concept_sets
kg_triples = {}

for i, concept_set in enumerate(concept_sets):
    print(i)
    terms = list(concept_set)
    top_k = 5
    pmids, distances, abstracts = retriever.search(" ".join(terms), top_k=top_k)

    for abstract in abstracts:
        response = extract_relationships(abstract, terms)
        triples = parse_triples(response, concept_set)

        for triple in triples:
            for concept in concept_set:
                if concept in triple:
                    if concept not in kg_triples:
                        kg_triples[concept] = []
                    kg_triples[concept].append(triple)
                    break

0


100%|██████████| 147/147 [08:14<00:00,  3.36s/it]


[('hydrogen ion concentration', 'affects', 'ulcerogenesis'), ('antacids', 'used for', 'prevention of stress ulcers'), ('H(2) receptor blocking agents', 'used for', 'treatment of stress ulcers'), ('acid', 'induces', 'gastric mucosal ulcerogenesis'), ('mucosal ischemia', 'contributes to', 'ulcerogenesis'), ('bile acid', 'induces', 'gastric mucosal ulcerogenesis'), ('H(2) receptor blocking agents', 'function as', 'prophylaxis against stress ulcer')]
[('prednisone', 'used for', "active Crohn's disease"), ('sulfasalazine', 'used for', "active Crohn's disease"), ('azathioprine', 'tested for', "Crohn's disease"), ('patients with colonic involvement', 'responsive to', 'sulfasalazine'), ('patients with small bowel involvement', 'responsive to', 'prednisone'), ('drug therapy', 'affects', 'subsequent response'), ("quiescent Crohn's disease", 'not prevented by', 'prednisone'), ("quiescent Crohn's disease", 'not prevented by', 'sulfasalazine'), ("quiescent Crohn's disease", 'not prevented by', 'aza

100%|██████████| 147/147 [08:06<00:00,  3.31s/it]


[]
[]
[('renal carcinoma', 'diagnosis involves', 'intravenous urography'), ('renal carcinoma', 'diagnosis involves', 'sonography'), ('renal carcinoma', 'diagnosis involves', 'computer-tomography'), ('therapy', 'includes', 'radical nephrectomy'), ('radical nephrectomy', 'improves', '5-year survival rate'), ('extracorporeal surgery', 'used in', 'single kidney patients'), ('embolization', 'used for', 'therapeutic management in progressed renal carcinoma'), ('irradiation', 'does not improve', 'renal carcinoma survival'), ('cytostatic therapy', 'does not improve', 'renal carcinoma survival'), ('early diagnosis', 'is crucial due to', 'significance of symptoms')]
[('phenacetin-containing analgesics', 'associated with', 'urorenal disorders'), ('increased intake of phenacetin-containing analgesics', 'leads to', 'higher incidence of raised serum-creatinine'), ('study group', 'comprised of', '623 employed Swiss women'), ('control group', 'showed', 'no intake of phenacetin-containing analgesics'),

...

100%|██████████| 147/147 [08:32<00:00,  3.48s/it]


[('saphenous vein graft', 'used in', 'coronary artery bypass'), ('anesthetic technique', 'was', 'neuroleptanalgesia'), ('postoperative care', 'included', 'standard water and electrolyte balance'), ('postoperative care', 'included', 'prophylactic antibiotics'), ('treatment', 'included', 'vasodilators'), ('vasodilators', 'mainly', 'IV nitroglycerine'), ('treatment', 'included', 'catecholamines'), ('treatment', 'included', 'anti-arrhythmic agents'), ('treatment', 'included', 'intra-aortic counter pulse balloon'), ('study', 'evaluated', 'effects of treatment on electrocardiographic abnormalities')]
[('postoperative', 'refers to', 'complications following surgery'), ('neuroleptanalgesia', 'is suitable for', 'aorto-coronary bypass surgery'), ('neuroleptanalgesia', 'decreases incidence of', 'hypertensive crises'), ('cardioplegia', 'decreases incidence of', 'postoperative complications'), ('aorto-coronary bypass surgery', 'involves', 'anaesthetic protocol'), ('postoperative', 'includes', 'comp

100%|██████████| 147/147 [08:31<00:00,  3.48s/it]


[('beta blockade', 'is related to', 'long term beta-adrenoceptor blocking drugs'), ('beta blockade', 'occurs during', 'anaesthesia'), ('beta blockade', 'is present in', 'coronary artery surgery'), ('patients', 'were anaesthetised for', 'aorto-coronary bypass operations'), ('beta blockade', 'diminishes', 'undesirable cardiovascular responses to laryngoscopy and tracheal intubation'), ('beta blockade', 'reduces', 'need for peripheral vasodilators'), ('coronary artery disease', 'is a condition in', 'patients undergoing surgery')]
[('bunitrolol', 'is a', 'beta-blocking agent'), ('bunitrolol', 'effective in', 'normalizing myocardial lactate extraction ratio'), ('bunitrolol', 'reduces', 'arterial non-esterified fatty acids (NEFA) level'), ('bunitrolol', 'reduces', 'myocardial NEFA extraction'), ('beta-blocking agent', 'effective in', 'treatment of ischemic heart disease'), ('myocardial anaerobic metabolic pattern', 'induced by', 'isoproterenol infusion'), ('beta-blocking agent', 'action', 'r

 83%|████████▎ | 122/147 [07:06<01:26,  3.48s/it]

In [19]:
kg_triples_dict = build_kg_for_concepts(filtered_concept_sets)

    # Save the KG triples to a JSON file
with open("kg_from_pubmed_DN.json", "w") as f:
    json.dump(merged, f, indent=4)

print(f"Number of filtered concept sets: {len(filtered_concept_sets)}")
print(f"Number of concepts with triples: {len(kg_triples_dict)}")

Number of filtered concept sets: 9583


NameError: name 'kg_triples_dict' is not defined